# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 2 2022 12:52PM,249901,10,8724704,Yusen – 3D Matrix,Released
1,Nov 2 2022 12:48PM,249900,20,ATR-I000018,"HVL, LLC dba Atrium Innovations",Released
2,Nov 2 2022 12:47PM,249899,19,8724698,"Emersa Waterbox, LLC",Released
3,Nov 2 2022 12:47PM,249899,19,8724703,"Emersa Waterbox, LLC",Released
4,Nov 2 2022 12:45PM,249898,10,8724695,Yusen – 3D Matrix,Released
5,Nov 2 2022 12:34PM,249897,21,415567883,"NBTY Global, Inc.",Released
6,Nov 2 2022 12:32PM,249896,12,HH-37926,Hush Hush,Released
7,Nov 2 2022 12:32PM,249896,12,HH-37927,Hush Hush,Released
8,Nov 2 2022 12:32PM,249896,12,HH-37928,Hush Hush,Released
9,Nov 2 2022 12:32PM,249896,12,HH-37929,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,249897,Released,1
44,249898,Released,1
45,249899,Released,2
46,249900,Released,1
47,249901,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249897,NaN,NaN,1.0
249898,NaN,NaN,1.0
249899,NaN,NaN,2.0
249900,NaN,NaN,1.0
249901,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249828,0.0,0.0,1.0
249837,0.0,0.0,2.0
249846,2.0,6.0,7.0
249847,4.0,7.0,2.0
249855,0.0,3.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249828,0,0,1
249837,0,0,2
249846,2,6,7
249847,4,7,2
249855,0,3,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249828,0,0,1
1,249837,0,0,2
2,249846,2,6,7
3,249847,4,7,2
4,249855,0,3,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249828,,,1
1,249837,,,2
2,249846,2,6,7
3,249847,4,7,2
4,249855,,3,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 2 2022 12:52PM,249901,10,Yusen – 3D Matrix
1,Nov 2 2022 12:48PM,249900,20,"HVL, LLC dba Atrium Innovations"
2,Nov 2 2022 12:47PM,249899,19,"Emersa Waterbox, LLC"
4,Nov 2 2022 12:45PM,249898,10,Yusen – 3D Matrix
5,Nov 2 2022 12:34PM,249897,21,"NBTY Global, Inc."
6,Nov 2 2022 12:32PM,249896,12,Hush Hush
11,Nov 2 2022 12:30PM,249895,21,"NBTY Global, Inc."
12,Nov 2 2022 12:23PM,249894,21,"NBTY Global, Inc."
13,Nov 2 2022 12:20PM,249893,21,"NBTY Global, Inc."
14,Nov 2 2022 12:20PM,249892,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 2 2022 12:52PM,249901,10,Yusen – 3D Matrix,,,1
1,Nov 2 2022 12:48PM,249900,20,"HVL, LLC dba Atrium Innovations",,,1
2,Nov 2 2022 12:47PM,249899,19,"Emersa Waterbox, LLC",,,2
3,Nov 2 2022 12:45PM,249898,10,Yusen – 3D Matrix,,,1
4,Nov 2 2022 12:34PM,249897,21,"NBTY Global, Inc.",,,1
5,Nov 2 2022 12:32PM,249896,12,Hush Hush,,,5
6,Nov 2 2022 12:30PM,249895,21,"NBTY Global, Inc.",,,1
7,Nov 2 2022 12:23PM,249894,21,"NBTY Global, Inc.",,,1
8,Nov 2 2022 12:20PM,249893,21,"NBTY Global, Inc.",,,1
9,Nov 2 2022 12:20PM,249892,10,Eywa Pharma Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 12:52PM,249901,10,Yusen – 3D Matrix,1,,
1,Nov 2 2022 12:48PM,249900,20,"HVL, LLC dba Atrium Innovations",1,,
2,Nov 2 2022 12:47PM,249899,19,"Emersa Waterbox, LLC",2,,
3,Nov 2 2022 12:45PM,249898,10,Yusen – 3D Matrix,1,,
4,Nov 2 2022 12:34PM,249897,21,"NBTY Global, Inc.",1,,
5,Nov 2 2022 12:32PM,249896,12,Hush Hush,5,,
6,Nov 2 2022 12:30PM,249895,21,"NBTY Global, Inc.",1,,
7,Nov 2 2022 12:23PM,249894,21,"NBTY Global, Inc.",1,,
8,Nov 2 2022 12:20PM,249893,21,"NBTY Global, Inc.",1,,
9,Nov 2 2022 12:20PM,249892,10,Eywa Pharma Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 12:52PM,249901,10,Yusen – 3D Matrix,1,,
1,Nov 2 2022 12:48PM,249900,20,"HVL, LLC dba Atrium Innovations",1,,
2,Nov 2 2022 12:47PM,249899,19,"Emersa Waterbox, LLC",2,,
3,Nov 2 2022 12:45PM,249898,10,Yusen – 3D Matrix,1,,
4,Nov 2 2022 12:34PM,249897,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 12:52PM,249901,10,Yusen – 3D Matrix,1,NaN,NaN
1,Nov 2 2022 12:48PM,249900,20,"HVL, LLC dba Atrium Innovations",1,NaN,NaN
2,Nov 2 2022 12:47PM,249899,19,"Emersa Waterbox, LLC",2,NaN,NaN
3,Nov 2 2022 12:45PM,249898,10,Yusen – 3D Matrix,1,NaN,NaN
4,Nov 2 2022 12:34PM,249897,21,"NBTY Global, Inc.",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 2 2022 12:52PM,249901,10,Yusen – 3D Matrix,1,0.0,0.0
1,Nov 2 2022 12:48PM,249900,20,"HVL, LLC dba Atrium Innovations",1,0.0,0.0
2,Nov 2 2022 12:47PM,249899,19,"Emersa Waterbox, LLC",2,0.0,0.0
3,Nov 2 2022 12:45PM,249898,10,Yusen – 3D Matrix,1,0.0,0.0
4,Nov 2 2022 12:34PM,249897,21,"NBTY Global, Inc.",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3498160,76,38.0,6.0
12,249896,5,0.0,0.0
15,999427,18,12.0,11.0
16,499772,2,0.0,0.0
19,749671,4,4.0,0.0
20,249900,1,0.0,0.0
21,2498892,10,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3498160,76,38.0,6.0
1,12,249896,5,0.0,0.0
2,15,999427,18,12.0,11.0
3,16,499772,2,0.0,0.0
4,19,749671,4,4.0,0.0
5,20,249900,1,0.0,0.0
6,21,2498892,10,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,76,38.0,6.0
1,12,5,0.0,0.0
2,15,18,12.0,11.0
3,16,2,0.0,0.0
4,19,4,4.0,0.0
5,20,1,0.0,0.0
6,21,10,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,76.0
1,12,Released,5.0
2,15,Released,18.0
3,16,Released,2.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,6.0,0.0,11.0,0.0,0.0,0.0,0.0
Executing,38.0,0.0,12.0,0.0,4.0,0.0,0.0
Released,76.0,5.0,18.0,2.0,4.0,1.0,10.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,6.0,0.0,11.0,0.0,0.0,0.0,0.0
1,Executing,38.0,0.0,12.0,0.0,4.0,0.0,0.0
2,Released,76.0,5.0,18.0,2.0,4.0,1.0,10.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,6.0,0.0,11.0,0.0,0.0,0.0,0.0
1,Executing,38.0,0.0,12.0,0.0,4.0,0.0,0.0
2,Released,76.0,5.0,18.0,2.0,4.0,1.0,10.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()